<a href="https://colab.research.google.com/github/federicoding/TrainingGeneral/blob/main/MiniExam01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mini Examen

Ceci est un raisonnement type pour un mini-examen

Ici c'est basé sur heart_disease.csv.

Même si, ici, je passe sur le chargement des données.